In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from datetime import datetime
from datetime import timedelta 
import time
import numpy as np

## 先連好資料庫

In [2]:
#這是折床機台的東西

try:
    cnx = mysql.connector.connect(host='',
                                  user='',
                                  passwd='',
                                  database='welding_report',
                                  )

    if cnx.is_connected():
        # 顯示資料庫版本
        db_Info = cnx.get_server_info()
        cursor = cnx.cursor(named_tuple=True)
        print('svfs database version:',db_Info)

except Error as e:
    print("資料庫連接失敗：", e)


sql = "SELECT worker_id, parent_id, result_start_time, result_end_time, TIMEDIFF(result_end_time,result_start_time) as `delta_time`,expect_amount, result_amount\
       FROM welding_log\
       WHERE result_start_time >= '2020-05-31 00:00:00' and result_end_time <='2020-08-01 23:59:59' ORDER BY result_start_time ASC"



#=============================================================#
#這邊是拿工人的名字

try:
    cnx_name = mysql.connector.connect(host='59.120.87.160',
                                  user='smb',
                                  passwd='Csie3621',
                                  database='welding_report',
                                  )

    if cnx_name.is_connected():
        # 顯示資料庫版本
        db_Info = cnx_name.get_server_info()
        cursor_name = cnx_name.cursor(named_tuple=True)
        print('welding_report database version:',db_Info)

except Error as e:
    print("資料庫連接失敗：", e)

sql_name = "SELECT username , realname \
            FROM weldworker"

svfs database version: 5.7.29-32
welding_report database version: 5.7.29-32


## 這邊是處理它的對應週數、週總耗時

#### (實際的耗時的週總耗時之後會才會算，現在先寫沒關係，而且下面這個拿來算標準工時的週總耗時同樣OK，等下再拿來用一次，因為還沒MERGE，反正這個之後一定是第一步)

In [3]:
cursor.execute(sql)
parents_log = cursor.fetchall()  #先獲取焊接報表DATA
parents_log

#============================

week_number=23 #從哪一周開始算
count_time = timedelta()  #耗時加總用
index_list = [] #把這一週的東西新增進去

real_parents_log=[]

for number,index in enumerate(parents_log):

    if week_number == int(index.result_start_time.strftime("%U"))+1:  #第一次進的會是23
        count_time += index[4] #裡面的所有耗時加總
        index_list.append(index) #把屬於23週的東西新增到List

    elif week_number != int(index.result_start_time.strftime("%U"))+1: #當它不再=23，應該說當他換週時:
        for index_save in index_list: #剛剛新增進去list裡面的東西
            real_parents_log.append([index_save[0],
                                    index_save[1],
                                    int(index_save.result_start_time.strftime("%U"))+1,
                                    count_time,
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    ]) #list裡面的東西
            
        count_time = timedelta() #重畳
        index_list = [] #重畳
        week_number = int(index.result_start_time.strftime("%U"))+1 #因為跳到下一週了，它本身會進入elif，這邊的週數也變成了那一週
        
        count_time += index[4] #裡面的所有耗時加總
        index_list.append(index) #把屬於下一週的東西新增到List
        
    if number==len(parents_log)-1: #當它來到最後一行時
        for index_save in index_list:
            real_parents_log.append([index_save[0],
                                    index_save[1],
                                    int(index_save.result_start_time.strftime("%U"))+1,
                                    count_time,
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    ])#list裡面的東西

## 加上工人的名字

##### 1 比較快的方法，一次從資料庫撈全部，然後再用pd merge，減少每次for迴圈都跑一次sql的東西

In [4]:
cursor_name.execute(sql_name)  
data_name = cursor_name.fetchall()
for i ,q in enumerate(data_name):
    print(i,q)

0 Row(username='111', realname=None)
1 Row(username='admin', realname=None)
2 Row(username='BENS', realname=None)
3 Row(username='MD001', realname='謝真斌')
4 Row(username='MD002', realname='李查')
5 Row(username='MD003', realname='林景南')
6 Row(username='MD004', realname='阮海運')
7 Row(username='MD009', realname='阮刻義')
8 Row(username='MD012', realname='阮文山')
9 Row(username='MD005', realname='范文團')
10 Row(username='MD006', realname='恩文中')
11 Row(username='MD007', realname='劉炳輝')
12 Row(username='MD008', realname='阮光榮')
13 Row(username='MD011', realname='黎得皇')
14 Row(username='MD013', realname='黃木賢')
15 Row(username='MD014', realname=None)
16 Row(username='MD015', realname=None)
17 Row(username='MC001', realname=None)
18 Row(username='MD010', realname='林友亮')
19 Row(username='MC002', realname=None)
20 Row(username='MC003', realname=None)
21 Row(username='MC004', realname=None)
22 Row(username='MC005', realname=None)


##### 2 用成2個Dataframe，再merge

In [5]:
#缺工人名稱的焊接報表
df_parents_log = pd.DataFrame(real_parents_log)
df_parents_log.columns=['工人編號','母件料號','週數','週總耗時', '起始時間', '結束時間', '耗時','預期完成數量', '實際生產數量']

#從weld_worker去撈工人名稱
df_name = pd.DataFrame(data_name)
df_name.columns=['工人編號','工人']

In [6]:
data=pd.merge(df_parents_log, df_name, how='left')  
#直接合起來最快
data

,工人編號,母件料號,週數,週總耗時,起始時間,結束時間,耗時,預期完成數量,實際生產數量,工人
0,MD003,2S80D1038C2,23,153 days 00:18:15,2020-06-02 11:50:56,2020-06-04 08:54:31,1 days 21:03:35,1,1,林景南
1,MD002,PU650FU3069,23,153 days 00:18:15,2020-06-02 11:56:15,2020-06-08 11:08:28,5 days 23:12:13,1,1,李查
2,MD009,PV116FU3002,23,153 days 00:18:15,2020-06-02 11:58:37,2020-06-03 10:30:49,0 days 22:32:12,1,1,阮刻義
3,,2S80D1038C2,23,153 days 00:18:15,2020-06-02 13:05:37,2020-06-02 13:31:29,0 days 00:25:52,1,1,NaN
4,,PU650FU3069,23,153 days 00:18:15,2020-06-02 13:22:52,2020-06-02 14:28:05,0 days 01:05:13,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...
2155,MD011,PV860FU0305,31,37 days 14:23:58,2020-08-01 13:46:41,2020-08-01 14:15:39,0 days 00:28:58,2,2,黎得皇
2156,MD011,PV860FU0139,31,37 days 14:23:58,2020-08-01 14:16:00,2020-08-01 14:49:32,0 days 00:33:32,2,2,黎得皇
2157,MD012,PM010E56,31,37 days 14:23:58,2020-08-01 14:24:29,2020-08-01 15:56:48,0 days 01:32:19,2,2,阮文山
2158,MD011,PV860FU0309,31,37 days 14:23:58,2020-08-01 14:49:48,2020-08-01 15:10:12,0 days 00:20:24,2,2,黎得皇


## 再加上母件的標準工時，現在先用這個算

##### 1 匯入標準工時EXCEL

In [7]:
#讀取標準工時表
standard_time_table = pd.read_csv('real_standard_time.csv') 
standard_time_table=standard_time_table[['product_no', 'time']] #只拿我要的COLUMNS
standard_time_table.columns = ['母件料號', '標準工時']

# 缺標準工時的DATA已經有了 : data

##### 2 合在一起

In [8]:
data=pd.merge(data, standard_time_table, how='left')  
#直接合起來最快
data

,工人編號,母件料號,週數,週總耗時,起始時間,結束時間,耗時,預期完成數量,實際生產數量,工人,標準工時
0,MD003,2S80D1038C2,23,153 days 00:18:15,2020-06-02 11:50:56,2020-06-04 08:54:31,1 days 21:03:35,1,1,林景南,35
1,MD002,PU650FU3069,23,153 days 00:18:15,2020-06-02 11:56:15,2020-06-08 11:08:28,5 days 23:12:13,1,1,李查,20
2,MD009,PV116FU3002,23,153 days 00:18:15,2020-06-02 11:58:37,2020-06-03 10:30:49,0 days 22:32:12,1,1,阮刻義,480
3,,2S80D1038C2,23,153 days 00:18:15,2020-06-02 13:05:37,2020-06-02 13:31:29,0 days 00:25:52,1,1,NaN,35
4,,PU650FU3069,23,153 days 00:18:15,2020-06-02 13:22:52,2020-06-02 14:28:05,0 days 01:05:13,1,1,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...
2155,MD011,PV860FU0305,31,37 days 14:23:58,2020-08-01 13:46:41,2020-08-01 14:15:39,0 days 00:28:58,2,2,黎得皇,10
2156,MD011,PV860FU0139,31,37 days 14:23:58,2020-08-01 14:16:00,2020-08-01 14:49:32,0 days 00:33:32,2,2,黎得皇,10
2157,MD012,PM010E56,31,37 days 14:23:58,2020-08-01 14:24:29,2020-08-01 15:56:48,0 days 01:32:19,2,2,阮文山,35
2158,MD011,PV860FU0309,31,37 days 14:23:58,2020-08-01 14:49:48,2020-08-01 15:10:12,0 days 00:20:24,2,2,黎得皇,10


##### 3 轉回LIST操作

In [9]:
data_transition=np.array(data)
data_transition=data_transition.tolist()


data_list=[]

for i in data_transition:
    if type(i[10]) == str:
        data_list.append([i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],int(i[10])])
    else:
        data_list.append([i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],0]) #把NAN值直接變0 ，不然後面根本運算不了


##### 4 去算它的標準工時週總耗時

In [10]:
week_number=23 #從哪一周開始算
count_time = 0  #耗時加總用
index_list = [] #把這一週的東西新增進去

real_data=[]

for number,index in enumerate(data_list):

    if week_number == index[2]:  #第一次進的會是23
        count_time += index[10] #裡面的所有耗時加總
        index_list.append(index) #把屬於23週的東西新增到List

    elif week_number != index[2]: #當它不再=23，應該說當他換週時:
        for index_save in index_list: #剛剛新增進去list裡面的東西
            real_data.append([index_save[0],
                                    index_save[1],
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    index_save[7],
                                    index_save[8],
                                    index_save[9],
                                    index_save[10],
                                    count_time
                                    ]) #list裡面的東西
            
        count_time = 0 #重畳
        index_list = [] #重畳
        week_number = index[2] #因為跳到下一週了，它本身會進入elif，這邊的週數也變成了那一週
        
        count_time += index[10] #裡面的所有耗時加總
        index_list.append(index) #把屬於下一週的東西新增到List
        
    if number==len(data_list)-1: #當它來到最後一行時
        for index_save in index_list:
            real_data.append([index_save[0],
                                    index_save[1],
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    index_save[7],
                                    index_save[8],
                                    index_save[9],
                                    index_save[10],
                                    count_time
                                    ])#list裡面的東西

In [11]:
real_data=pd.DataFrame(real_data)
real_data.columns=['工人編號','母件料號','週數','週總耗時', '起始時間', '結束時間', '耗時','預期完成數量', '實際生產數量','工人','標準工時','週總標準工時耗時']
real_data

,工人編號,母件料號,週數,週總耗時,起始時間,結束時間,耗時,預期完成數量,實際生產數量,工人,標準工時,週總標準工時耗時
0,MD003,2S80D1038C2,23,153 days 00:18:15,2020-06-02 11:50:56,2020-06-04 08:54:31,1 days 21:03:35,1,1,林景南,35,10080
1,MD002,PU650FU3069,23,153 days 00:18:15,2020-06-02 11:56:15,2020-06-08 11:08:28,5 days 23:12:13,1,1,李查,20,10080
2,MD009,PV116FU3002,23,153 days 00:18:15,2020-06-02 11:58:37,2020-06-03 10:30:49,0 days 22:32:12,1,1,阮刻義,480,10080
3,,2S80D1038C2,23,153 days 00:18:15,2020-06-02 13:05:37,2020-06-02 13:31:29,0 days 00:25:52,1,1,NaN,35,10080
4,,PU650FU3069,23,153 days 00:18:15,2020-06-02 13:22:52,2020-06-02 14:28:05,0 days 01:05:13,1,1,NaN,20,10080
...,...,...,...,...,...,...,...,...,...,...,...,...
2155,MD011,PV860FU0305,31,37 days 14:23:58,2020-08-01 13:46:41,2020-08-01 14:15:39,0 days 00:28:58,2,2,黎得皇,10,19318
2156,MD011,PV860FU0139,31,37 days 14:23:58,2020-08-01 14:16:00,2020-08-01 14:49:32,0 days 00:33:32,2,2,黎得皇,10,19318
2157,MD012,PM010E56,31,37 days 14:23:58,2020-08-01 14:24:29,2020-08-01 15:56:48,0 days 01:32:19,2,2,阮文山,35,19318
2158,MD011,PV860FU0309,31,37 days 14:23:58,2020-08-01 14:49:48,2020-08-01 15:10:12,0 days 00:20:24,2,2,黎得皇,10,19318


## 薪資表

##### 1 匯入薪資表

In [12]:
wage_table = pd.read_excel('薪資統計FOR成本.xlsx')  #讀取薪資表
wage_table

,日期,星期,週數,雷射薪資,折床薪資,焊接薪資,噴漆薪資,組立薪資
0,2020-01-01,星期三,1,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,星期四,1,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,星期五,1,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,星期六,1,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,星期日,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
361,2020-12-27,星期日,53,NaN,NaN,NaN,NaN,NaN
362,2020-12-28,星期一,53,NaN,NaN,NaN,NaN,NaN
363,2020-12-29,星期二,53,NaN,NaN,NaN,NaN,NaN
364,2020-12-30,星期三,53,NaN,NaN,NaN,NaN,NaN


##### 2 把它groupby，再單獨變成一個dataframe

In [13]:
agg=wage_table.groupby('週數')['焊接薪資'].agg(np.sum)
real_wage_table=pd.DataFrame(agg)
real_wage_table.reset_index(inplace=True)
real_wage_table

,週數,焊接薪資
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
5,6,0.0
6,7,0.0
7,8,0.0
8,9,0.0
9,10,0.0


##### 3 再merge

In [14]:
welding=pd.merge(real_data,real_wage_table, how='left')  
welding

,工人編號,母件料號,週數,週總耗時,起始時間,結束時間,耗時,預期完成數量,實際生產數量,工人,標準工時,週總標準工時耗時,焊接薪資
0,MD003,2S80D1038C2,23,153 days 00:18:15,2020-06-02 11:50:56,2020-06-04 08:54:31,1 days 21:03:35,1,1,林景南,35,10080,118861.0
1,MD002,PU650FU3069,23,153 days 00:18:15,2020-06-02 11:56:15,2020-06-08 11:08:28,5 days 23:12:13,1,1,李查,20,10080,118861.0
2,MD009,PV116FU3002,23,153 days 00:18:15,2020-06-02 11:58:37,2020-06-03 10:30:49,0 days 22:32:12,1,1,阮刻義,480,10080,118861.0
3,,2S80D1038C2,23,153 days 00:18:15,2020-06-02 13:05:37,2020-06-02 13:31:29,0 days 00:25:52,1,1,NaN,35,10080,118861.0
4,,PU650FU3069,23,153 days 00:18:15,2020-06-02 13:22:52,2020-06-02 14:28:05,0 days 01:05:13,1,1,NaN,20,10080,118861.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,MD011,PV860FU0305,31,37 days 14:23:58,2020-08-01 13:46:41,2020-08-01 14:15:39,0 days 00:28:58,2,2,黎得皇,10,19318,105316.0
2156,MD011,PV860FU0139,31,37 days 14:23:58,2020-08-01 14:16:00,2020-08-01 14:49:32,0 days 00:33:32,2,2,黎得皇,10,19318,105316.0
2157,MD012,PM010E56,31,37 days 14:23:58,2020-08-01 14:24:29,2020-08-01 15:56:48,0 days 01:32:19,2,2,阮文山,35,19318,105316.0
2158,MD011,PV860FU0309,31,37 days 14:23:58,2020-08-01 14:49:48,2020-08-01 15:10:12,0 days 00:20:24,2,2,黎得皇,10,19318,105316.0


##### 4. 用加進來的薪資算出總成本，順便處理一下生產為0的值

In [15]:
count=0

divided_cost=[]

for index ,row in welding.iterrows():
    divided_cost.append(row[12]*row[10]/row[11])
    
    #這邊順便處理一下實際生產數量如果是0的話...
    if row[8]<=0:  #這個地方處理的是'如果完成數量為0',前面用的「數量」就代表它的完成數量，如果也是0的話就只能是1了
        welding.iloc[index,8]=row[7]
        count+=1

print(count)

welding['總成本']=divided_cost
welding

189


,工人編號,母件料號,週數,週總耗時,起始時間,結束時間,耗時,預期完成數量,實際生產數量,工人,標準工時,週總標準工時耗時,焊接薪資,總成本
0,MD003,2S80D1038C2,23,153 days 00:18:15,2020-06-02 11:50:56,2020-06-04 08:54:31,1 days 21:03:35,1,1,林景南,35,10080,118861.0,412.711806
1,MD002,PU650FU3069,23,153 days 00:18:15,2020-06-02 11:56:15,2020-06-08 11:08:28,5 days 23:12:13,1,1,李查,20,10080,118861.0,235.835317
2,MD009,PV116FU3002,23,153 days 00:18:15,2020-06-02 11:58:37,2020-06-03 10:30:49,0 days 22:32:12,1,1,阮刻義,480,10080,118861.0,5660.047619
3,,2S80D1038C2,23,153 days 00:18:15,2020-06-02 13:05:37,2020-06-02 13:31:29,0 days 00:25:52,1,1,NaN,35,10080,118861.0,412.711806
4,,PU650FU3069,23,153 days 00:18:15,2020-06-02 13:22:52,2020-06-02 14:28:05,0 days 01:05:13,1,1,NaN,20,10080,118861.0,235.835317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,MD011,PV860FU0305,31,37 days 14:23:58,2020-08-01 13:46:41,2020-08-01 14:15:39,0 days 00:28:58,2,2,黎得皇,10,19318,105316.0,54.517031
2156,MD011,PV860FU0139,31,37 days 14:23:58,2020-08-01 14:16:00,2020-08-01 14:49:32,0 days 00:33:32,2,2,黎得皇,10,19318,105316.0,54.517031
2157,MD012,PM010E56,31,37 days 14:23:58,2020-08-01 14:24:29,2020-08-01 15:56:48,0 days 01:32:19,2,2,阮文山,35,19318,105316.0,190.809608
2158,MD011,PV860FU0309,31,37 days 14:23:58,2020-08-01 14:49:48,2020-08-01 15:10:12,0 days 00:20:24,2,2,黎得皇,10,19318,105316.0,54.517031


## 接下來就是算每個母件的單一焊接成本

In [16]:
true_welding_cost=[]

grouped = welding.groupby('母件料號') #用groupby的方法處理
agg = grouped['實際生產數量','總成本'].agg(np.sum)
welding_cost = pd.DataFrame(agg)
welding_cost.reset_index(inplace=True)

for i,q in zip(welding_cost['總成本'],welding_cost['實際生產數量']):
    true_welding_cost.append(i/q)

welding_cost['母件焊接單一成本']=true_welding_cost
welding_cost

,母件料號,實際生產數量,總成本,母件焊接單一成本
0,0E1013F062A0,3,128.769317,42.923106
1,0Q0601F017A0,2,81.775546,40.887773
2,0QQ060F017A0,1,0.000000,0.000000
3,0V0814F020A0,3,128.769317,42.923106
4,1AK0F1015A1,4,155.878374,38.969593
...,...,...,...,...
1170,X8SQ348A,2,4245.035714,2122.517857
1171,X8SQ349A,1,2122.517857,2122.517857
1172,X8SQ372A,4,707.505952,176.876488
1173,X8SQ383A,4,48.534721,12.133680


In [17]:
welding_cost.to_csv('welding0531-0801.csv', index=False, mode='w', header=True, encoding='utf_8_sig')